In [44]:
import sqlalchemy
import pandas as pd
from sqlalchemy import create_engine
from pandasql import sqldf
from sqlalchemy.orm import sessionmaker
pysqldf = lambda q: sqldf(q, globals())

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',sep=",",header=None)
df.columns=['age','workclass','fnlwgt','education','educationNum','marital_status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','income']
# Using the create_engine module and establishing an sqlite3 session and connecting to adult.db database
engine = create_engine('sqlite:///adult.db')
# Using the to_sql function to convert our datframe to a table using our pre-defined engine 
df.to_sql('adulttabassign', engine)

In [74]:
con = engine.connect()
# Testing my session using a sample test query
test = con.execute("select * from adulttabassign limit 3;")
test.fetchall()

[(0, 39, ' State-gov', 77516, ' Bachelors', 13, ' Never-married', ' Adm-clerical', ' Not-in-family', ' White', ' Male', 2174, 0, 40, ' United-States', ' <=50K'),
 (1, 50, ' Self-emp-not-inc', 83311, 'HS-grad', 9, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' White', ' Male', 0, 0, 13, ' United-States', ' <=50K'),
 (2, 38, ' Private', 215646, ' HS-grad', 9, ' Divorced', ' Handlers-cleaners', ' Not-in-family', ' White', ' Male', 0, 0, 40, ' United-States', ' <=50K')]

In [64]:
# Performing a simple update on education column
upd = con.execute("update adulttabassign set education='HS-grad' where age=50 and fnlwgt='83311';")
# Selecting the updated row to see the changes
upd_row1 = con.execute("select * from adulttabassign where age=50 and fnlwgt='83311';")
upd_row1.fetchall()

[(1, 50, ' Self-emp-not-inc', 83311, 'HS-grad', 9, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' White', ' Male', 0, 0, 13, ' United-States', ' <=50K')]

In [65]:
# Performing a simple update on educationNum column to match its corresponding education
upd2 = con.execute("update adulttabassign set educationNum=9 where age=50 and fnlwgt='83311';")
# Selecting the updated row to see the changes
upd_row2 = con.execute("select * from adulttabassign where age=50 and fnlwgt='83311';")
upd_row2.fetchall()

[(1, 50, ' Self-emp-not-inc', 83311, 'HS-grad', 9, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' White', ' Male', 0, 0, 13, ' United-States', ' <=50K')]

In [48]:
# Sample delete operations based on a specific condition
dele = con.execute("delete from adulttabassign where age > 60;")
dele1 = con.execute("delete from adulttabassign where fnlwgt > 215646;")

In [49]:
# Commiting all my above transactions 
transaction = con.begin()
transaction.commit()

In [63]:
# Filtering based on marital status and relationship 
filter1 = con.execute("select * from adulttabassign where trim(marital_status) like 'Married%' and trim(relationship) = 'Husband';")
filter1.fetchall()

[(1, 50, ' Self-emp-not-inc', 83311, 'HS-grad', 9, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' White', ' Male', 0, 0, 13, ' United-States', ' <=50K'),
 (7, 52, ' Self-emp-not-inc', 209642, ' HS-grad', 9, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' White', ' Male', 0, 0, 45, ' United-States', ' >50K'),
 (9, 42, ' Private', 159449, ' Bachelors', 13, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' White', ' Male', 5178, 0, 40, ' United-States', ' >50K'),
 (11, 30, ' State-gov', 141297, ' Bachelors', 13, ' Married-civ-spouse', ' Prof-specialty', ' Husband', ' Asian-Pac-Islander', ' Male', 0, 0, 40, ' India', ' >50K'),
 (14, 40, ' Private', 121772, ' Assoc-voc', 11, ' Married-civ-spouse', ' Craft-repair', ' Husband', ' Asian-Pac-Islander', ' Male', 0, 0, 40, ' ?', ' >50K'),
 (18, 38, ' Private', 28887, ' 11th', 7, ' Married-civ-spouse', ' Sales', ' Husband', ' White', ' Male', 0, 0, 50, ' United-States', ' <=50K'),
 (20, 40, ' Private', 193524, ' Doctorate

In [66]:
# Filtering based on age range between 30 and 40
filter2 = con.execute("select * from adulttabassign where age between 30 and 40;")
filter2.fetchall()

[(0, 39, ' State-gov', 77516, ' Bachelors', 13, ' Never-married', ' Adm-clerical', ' Not-in-family', ' White', ' Male', 2174, 0, 40, ' United-States', ' <=50K'),
 (2, 38, ' Private', 215646, ' HS-grad', 9, ' Divorced', ' Handlers-cleaners', ' Not-in-family', ' White', ' Male', 0, 0, 40, ' United-States', ' <=50K'),
 (8, 31, ' Private', 45781, ' Masters', 14, ' Never-married', ' Prof-specialty', ' Not-in-family', ' White', ' Female', 14084, 0, 50, ' United-States', ' >50K'),
 (11, 30, ' State-gov', 141297, ' Bachelors', 13, ' Married-civ-spouse', ' Prof-specialty', ' Husband', ' Asian-Pac-Islander', ' Male', 0, 0, 40, ' India', ' >50K'),
 (13, 32, ' Private', 205019, ' Assoc-acdm', 12, ' Never-married', ' Sales', ' Not-in-family', ' Black', ' Male', 0, 0, 50, ' United-States', ' <=50K'),
 (14, 40, ' Private', 121772, ' Assoc-voc', 11, ' Married-civ-spouse', ' Craft-repair', ' Husband', ' Asian-Pac-Islander', ' Male', 0, 0, 40, ' ?', ' >50K'),
 (17, 32, ' Private', 186824, ' HS-grad', 9,

In [72]:
# Realization of group by() and order by() function
order1 = con.execute("select age,education,relationship,count(*) from adulttabassign group by education order by 1 desc;")
order1.fetchall()

[(60, ' 9th', ' Not-in-family', 279),
 (58, ' 11th', ' Husband', 711),
 (50, 'HS-grad', ' Husband', 1),
 (49, ' 12th', ' Other-relative', 259),
 (45, ' 5th-6th', ' Husband', 127),
 (43, ' Assoc-voc', ' Husband', 920),
 (43, ' Some-college', ' Husband', 4695),
 (38, ' Bachelors', ' Unmarried', 3536),
 (37, ' Assoc-acdm', ' Not-in-family', 706),
 (37, ' Prof-school', ' Husband', 362),
 (36, ' 7th-8th', ' Husband', 292),
 (36, ' Preschool', ' Not-in-family', 20),
 (34, ' Doctorate', ' Husband', 239),
 (32, ' 10th', ' Husband', 550),
 (32, ' Masters', ' Not-in-family', 1179),
 (23, ' 1st-4th', ' Wife', 65),
 (22, ' HS-grad', ' Own-child', 6669)]

In [73]:
# Realization of average function and grouping it by education
order2 = con.execute("select avg(age),education from adulttabassign group by education;")
order2.fetchall()

[(34.78363636363636, ' 10th'),
 (30.461322081575247, ' 11th'),
 (30.26254826254826, ' 12th'),
 (43.4, ' 1st-4th'),
 (41.55118110236221, ' 5th-6th'),
 (42.48287671232877, ' 7th-8th'),
 (37.77777777777778, ' 9th'),
 (36.596317280453256, ' Assoc-acdm'),
 (37.24673913043478, ' Assoc-voc'),
 (37.29864253393665, ' Bachelors'),
 (44.72384937238494, ' Doctorate'),
 (37.41685410106463, ' HS-grad'),
 (42.31128074639525, ' Masters'),
 (40.8, ' Preschool'),
 (41.67403314917127, ' Prof-school'),
 (34.580404685835994, ' Some-college'),
 (50.0, 'HS-grad')]